In [2]:
from tree import tmtree
import numpy as np
import utils.sampling as smp
import tree.halomodule as hmo 
import load

from astropy.io import fits

wdir = '/home/hoseung/Work/data/05427/'

tt = fits.getdata(wdir + "GalaxyMaker/TMtree.fits", 1)


#tt = tmtree.load(base=wdir, filename="GalaxyMaker/TMtree.fits")


#h = halomodule.Halo(nout=187, base='/home/hoseung/Work/data/05427/', halofinder="HaloMaker", load=True)

#### Latest merger of each galaxy

In [3]:
tmtree.fix_nout(tt, 10, 187)
ifinal_gal = np.where(tt["NOUT"] == 187)[0]
final_gal = tt["HALNUM"][ifinal_gal]

#### extract merger trees of final galaxies

In [39]:
import tree.draw_merger_tree as dmt

def get_xarr(n):
    import numpy as np
    arr=[]
    a=0
    for i in range(n):
        a += (-1)**i * i
        arr.append(a)
    return np.asarray(arr)
       

def recursive_tree(idx, tt, nstep):
    prgs = tt[idx]["TREE"]
    #print("IDX:", idx, "prgs: ",prgs[:5], tt[idx]["NOUT"])
    prgs = prgs[prgs > 0]
    nprg = len(prgs)
    if nprg == 0:
        return
    if nstep == 0:
        return 
    else:
        if nprg > 1:
            print("Branch!", nprg)
#            dx *= 0.9
#            dxarr = np.arange(np.fix(-nprg/2), np.fix(nprg/2)+1)
#            xarr = dxarr[-nprg:]*dx + x0
            #xarr = get_xarr(nprg) * dx + x0
        for i in prgs:
#link_circle_up([x0, x], [y0, y0 + 1], m, ax)
                #print("Nstep", nstep, " prg ", i)
            recursive_tree(i, tt, nstep - 1)

def get_idx(tt, halnum, nout):
    """
    It's better to be included in tree object.
    """
    ind_now = np.where(tt["NOUT"] == nout)
    tt_sub = tt[ind_now]
    ind_hal = np.where(tt_sub["HALNUM"] == halnum)
    return tt_sub[ind_hal]["IDX"]


def last_merger(idx, tt, nstep, nstep_end):
    nprgs = 1
    while nprgs < 2:
        prgs = tt[idx]["TREE"]
        nprgs = len(prgs[prgs > 0])
        #print(nprgs)
        idx = prgs[prgs > 0]
        nstep -= 1
        if nstep == nstep_end:
            break
    return nstep


#### Find last merger event

In [40]:
last_mergers = np.zeros(len(final_gal), dtype=int)
for i, gal in enumerate(final_gal):
    idx0 = dmt.get_idx(tt, gal, 187)
    last_mergers[i] = last_merger(idx0, tt, 187, nstep_end=10)
    

In [41]:
last_mergers

array([185,  10,  10,  10,  10,  10, 159,  10,  10,  10,  10,  10,  10,
        10,  10,  10,  10,  10,  10,  10,  10,  10,  10,  10,  10,  10,
        10,  10,  10,  10,  10,  10,  10,  10,  10,  10,  10,  10,  10,
        10,  10,  10,  10,  10,  10,  10,  10,  10,  10,  10,  10,  10,
        10,  10,  10,  10,  10,  10,  10,  10,  10,  10,  10,  10,  10,
        10,  10,  10,  10,  10,  10,  10,  10,  10,  10,  10,  10,  10,
        10,  10,  10,  10,  10,  10,  10,  10,  10,  10,  10,  10,  10,
        10,  10,  10,  10,  10,  10,  10,  10,  10,  10,  10,  10,  10,
        10,  10,  10,  10,  10,  10,  10,  10,  10,  10,  10,  10,  10,
        10,  10,  10,  10,  10,  10,  10,  10,  10,  10,  10,  10,  10,
        10,  10,  10,  10,  10,  10,  10,  10,  10,  10,  10,  10,  10,
        10,  10,  10,  10,  10,  10,  10,  10,  10,  10,  10,  10,  10,
        10,  10,  10,  10,  10,  10,  10,  10,  10,  10,  10,  10,  10,
        10,  10,  10,  10,  10,  10,  10,  10,  10,  10,  10,  1

#### IT's merger tree is not satisfactory.
how about the original TreeMaker output?

In [37]:
wdir = '/home/hoseung/Work/data/05427/'
import numpy as np
from load import utils
f = open(wdir + '/GalaxyMaker/tree.dat', 'rb')
nsteps = utils.read_fortran(f, np.int32, n=1, check=True)[0]
nhals = utils.read_fortran(f, np.int32, n=2*nsteps, check=True)
nsub = nhals[nsteps:]
nhals = nhals[:nsteps]
aexps = utils.read_fortran(f, np.float32, n=nsteps, check=True)
omega_t = utils.read_fortran(f, np.float32, n=nsteps, check=True)
age_univ = utils.read_fortran(f, np.float32, n=nsteps, check=True)

In [39]:
idx=0
nhal_tot = sum(nhals) + sum(nsub)
istep = np.zeros(nhal_tot)

dtype = np.dtype([("hID", "i4"), ("bushID","i4"), ("st","i4"),
                  ("hh","i4", (5,)),
                  ("m","f4"), ("macc","f8"),
                  ("xp","f4", (3,)), ("vp","f4", (3,)), ("lp","f4", (3,)),
                  ("abc","f4", (4,)), ("energy","f4", (3,)), ("spin","f4"),
                  ("nb_fathers","i4"), ("nb_sons","i4")])

tt = np.array(nhal_tot, dtype=dtype)

for i in range(nsteps):
    nhalos = nhals[i] + nsub[i]
    for j in range(nhalos):
        tt[idx]["hID"] = utils.read_fortran(f, np.int32, n=1, check=True)[0]
        tt[idx]["bushID"] = utils.read_fortran(f, np.int32, n=1, check=True)[0]
        tt[idx]["st"] = utils.read_fortran(f, np.int32, n=1, check=True)[0]
        tt[idx]["hh"] = utils.read_fortran(f, np.int32, n=5, check=True)
        tt[idx]["m"]  = utils.read_fortran(f, np.float32, n=1, check=True)[0]
        tt[idx]["macc"] = utils.read_fortran(f, np.float64, n=1, check=True)[0]
        tt[idx]["xp"] = utils.read_fortran(f, np.float32, n=3, check=True)
        tt[idx]["vp"] = utils.read_fortran(f, np.float32, n=3, check=True)
        tt[idx]["lp"] = utils.read_fortran(f, np.float32, n=3, check=True)
        tt[idx]["abc"] = utils.read_fortran(f, np.float32, n=4, check=True)
        tt[idx]["energy"] = utils.read_fortran(f, np.float32, n=3, check=True)
        tt[idx]["spin"] = utils.read_fortran(f, np.float32, n=1, check=True)[0]

        nb_fathers = utils.read_fortran(f, np.int32, n=1, check=True)[0]
        tt[idx]["nb_fathers"] = nb_fathers
        if nb_fathers > 0:
            tmp_list_fathers = utils.read_fortran(f, np.int32, n=nb_fathers, check=True)
            tmp_mass_fathers = utils.read_fortran(f, np.float32, n=nb_fathers, check=True)

        nb_sons = utils.read_fortran(f, np.int32, n=1, check=True)
        if nb_sons > 0:
            tmp_list_sons = utils.read_fortran(f, np.int32, n=nb_sons, check=True)
        
        rvir, mvir, tvir, cvel = utils.read_fortran(f, np.float32, n=4, check=True)
        #rvir, mvir, tvir, cvel = tmp
        rho_0, r_c = utils.read_fortran(f, np.float32, n=2, check=True)
        
        
        # 
        
        
        istep[idx] = i
        idx+=1
        

In [40]:
istep

array([   0.,    0.,    0., ...,  177.,  177.,  177.])

In [13]:
utils.read_header(f, dtype=np.dtype([('nhal', 'i4', (nstep,2)),
         ('nsub', 'i4', (nstep,))]))
utils.read_header(f, dtype=np.dtype([('aexp', 'f8', (nstep,)),
         ('omega_t', 'f8', (nstep,)),
         ('age_univ', 'f8', (nstep,))]), check=True)

AttributeError: 'list' object has no attribute 'fields'

In [16]:
n_halos, n_subhals = utils.read_fortran(f, np.int32, n=1, check=True)[0]

array([178], dtype=int32)